# GDA Implementation.

Implement the Gaussian Discriminant Analysis (GDA) learning algorithm following the steps as discussed in class.

INSTRUCTION: Rename your notebook as: <br>
`firstName_LastName_Live_coding_GDA.ipynb`.

Notes: 
* Do not use any built-in functions to complete a task;
* Do not import additional libraries.

In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification

In [2]:
# Generate data
def generate_data():
  x, y = make_classification(n_samples= 1000, n_features=3, n_redundant=0, 
                           n_informative=3, random_state=1, 
                           n_clusters_per_class=1)
  
  return x,y

x,y= generate_data() # get data
print(x.shape, y.shape)

(1000, 3) (1000,)


In [3]:
def split_data(x,y, train_size= 0.8):
    # shuffle the data to randomize the train/test split
    data = np.hstack((x,y.reshape(-1,1)))
    np.random.shuffle(data)
   
    N = x.shape[0]
    X_train, y_train = data[:int(N*train_size), :-1], data[:int(N*train_size), -1]
    X_test, y_test = data[int(N*train_size):, :-1], data[int(N*train_size):, -1]

    return X_train, X_test, y_train, y_test


In [4]:
X_train, X_test, y_train, y_test= split_data(x,y) # split your data into x_train, x_test, y_train, y_test
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(800, 3) (800,) (200, 3) (200,)


In [5]:
def covariance(x, mu):

  # Easy way: cov= np.cov(x, rowvar=0) but do not use it. One can use it to assess his/her result.
  N, D = x.shape
  cov_matrix = np.zeros((D, D))
  
  for i in range(D):

    for j in range(D):
      # print(x.T[i], x.T[j])
      cov_matrix[i][j] = 1/(N-1)* np.sum((x.T[i] - mu[i])@(x.T[j] - mu[j]).T, axis = 0)
  return cov_matrix

In [6]:
covariance(X_train, [0,0,0])

array([[1.81072111, 0.00351152, 1.02391549],
       [0.00351152, 1.94629854, 0.0551522 ],
       [1.02391549, 0.0551522 , 1.81263367]])

In [7]:
np.cov(X_train, rowvar=0 )

array([[1.81071962, 0.0022824 , 1.02389112],
       [0.0022824 , 0.93446696, 0.03508599],
       [1.02389112, 0.03508599, 1.81223572]])

In [8]:
class GDA:
  def __init__(self):
    ## set mu, phi and sigma to None
    self.mu = None
    self.phi = None
    self.sigma = None
    
  def fit(self,x,y):
    k= len(np.unique(y)) # Number of class.
    d= x.shape[1]  # input dim
    m= x.shape[0] # Number of examples.
    classes = np.unique(y)
    
    ## Initialize mu, phi and sigma
    self.mu= np.zeros((k, d))#: kxd, i.e., each row contains an individual class mu.
    self.sigma= np.zeros((k, d, d))#: kxdxd, i.e., each row contains an individual class sigma.
    self.phi= np.zeros((d,1))# d-dimension

    ## START THE LEARNING: estimate mu, phi and sigma.

    for i, elt in enumerate(classes):
      idx = [(y == elt)]
      N = len(y[idx])
      self.phi[i] = N/m
      
      for j in range(d):
        self.mu[i][j] = np.sum((x[idx].T)[j])/ x[idx].shape[0]

      self.sigma[i] = covariance(x, self.mu[i])

    # return self.mu, self.phi, self.sigma
      


  def predict_proba(self,x):
    # reshape or flatt x.
    # x= x.reshape(-1, x.shape[1])
    d= x.shape[1]
    k_class= self.mu.shape[0] # Number of classes we have in our case it's k = 2
    probas = np.zeros((x.shape[0], self.mu.shape[0]))
    
    ## START THE LEARNING: estimate mu, phi and sigma.

    for k in range(k_class):
      det = np.linalg.det(self.sigma[k])
      inverse = np.linalg.inv(self.sigma[k])

      for j in range(len(x)):
        probas[j][k] = self.phi[k] * 1.0/(((2*math.pi)**d/2) * (det**0.5)) * np.exp(-0.5* (x[j] - self.mu[k]).T@(inverse)@(x[j] - self.mu[k]))

    return probas

        

  def predict(self,x):
    proba = self.predict_proba(x)
    k_class= self.mu.shape[0]
    y_pred = []

    for i in range(proba.shape[0]):
      y_pred.append(np.argmax(proba[i]))

    return y_pred

    
  
  def accuracy(self, y, ypreds):
    pres = np.mean(y==ypreds)*100

    return pres

In [9]:
model= GDA()
model.fit(X_train,y_train)

<ipython-input-8-cbd1cee72bd6>:23: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  N = len(y[idx])
<ipython-input-8-cbd1cee72bd6>:27: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  self.mu[i][j] = np.sum((x[idx].T)[j])/ x[idx].shape[0]


In [10]:
yproba= model.predict_proba(X_test)
yproba

array([[1.06859481e-04, 2.30298037e-04],
       [1.99679301e-05, 1.17513389e-05],
       [4.35037069e-04, 1.32400296e-03],
       [3.33652955e-04, 1.95231045e-04],
       [1.68285951e-04, 1.21055982e-04],
       [1.70455575e-03, 5.50831471e-04],
       [5.59392909e-04, 1.89912286e-03],
       [6.87528761e-04, 7.34874668e-04],
       [1.28507072e-03, 8.61747699e-04],
       [1.27611745e-04, 2.82204520e-04],
       [8.65954418e-05, 4.81712762e-04],
       [5.31291803e-04, 1.86378071e-03],
       [1.21142195e-03, 1.66278288e-03],
       [4.39032380e-04, 6.11115680e-04],
       [7.75071911e-04, 1.84102077e-03],
       [5.35214052e-04, 1.23460984e-03],
       [9.43794849e-05, 3.21851419e-04],
       [7.21240166e-04, 2.10638046e-03],
       [6.79393125e-04, 1.65072476e-03],
       [3.67770734e-04, 1.47275044e-03],
       [2.92669754e-04, 4.15655567e-04],
       [8.68405095e-04, 3.07370964e-04],
       [1.20667268e-03, 2.02003900e-03],
       [2.88992592e-04, 2.05018880e-04],
       [1.545223

In [11]:
y_test

array([1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0.,
       0., 1., 0., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1.,
       1., 0., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 1., 0., 1.,
       1., 1., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 1., 0., 1., 1., 1.,
       0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1.,
       1., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0.,
       1., 1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1.])

In [12]:
ypreds= model.predict(X_test)
ypreds

[1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1]

In [13]:
model.accuracy(y_test, ypreds)

96.0